In [3]:
import os
train_dir  = '../input/100-bird-species/train'
valid_dir ='../input/100-bird-species/valid'
test_dir ='../input/100-bird-species/test'
classes = os.listdir(train_dir)
classes

In [4]:
import torchvision.transforms as tt

transform = tt.Compose([tt.Resize((224, 224)), tt.ToTensor(),tt.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(train_dir, transform = transform)
valid_dataset = ImageFolder(valid_dir, transform = transform)
test_dataset = ImageFolder(test_dir, transform = transform)

from torch.utils.data.dataloader import DataLoader
batch_size = 32

train_dl = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=4, pin_memory=True)
valid_dl = DataLoader(valid_dataset, batch_size*2, num_workers=4, pin_memory=True)

classes = valid_dataset.classes

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
def show_example(data):
    [img, label] = data
    print(classes[label])
    plt.imshow(img.permute(1, 2, 0))
    
from random import randrange
image_number = randrange(20000)
show_example(train_dataset[image_number])

In [6]:
from torchvision.utils import make_grid
def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 12))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=8).permute(1, 2, 0))
        break

In [7]:
show_batch(train_dl)

In [14]:
from keras.models import Sequential
from keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Dropout, BatchNormalization
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(400,activation='softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ["accuracy"])
model.summary()

In [15]:
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator()
train_ds = generator.flow_from_directory(train_dir,target_size=(224, 224),batch_size=32)
h1=model.fit(train_ds,epochs= 15, batch_size=32)

In [37]:
from keras.models import load_model

model.save('my_model.h5')

In [39]:
h1=model.fit(train_ds,epochs= 15, batch_size=32)

In [40]:
h1=model.fit(train_ds,epochs= 5, batch_size=32)

In [41]:
h1=model.fit(train_ds,epochs= 3, batch_size=32)
model.save('my_model1.h5')

In [42]:
import os
os.listdir("./")

<a href="./my_model1.h5"> Download File </a>

In [68]:
test_ds = generator.flow_from_directory(test_dir,target_size=(224, 224),batch_size=32)
test_loss, test_acc = model.evaluate(test_ds, steps=50)
print('test acc: ', round(test_acc*100, 2),  '%')

In [1]:
import tensorflow
from tensorflow import keras
model = keras.models.load_model('../input/cnnbirds/cnnmodel.h5')
from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator()
train_ds = generator.flow_from_directory('../input/100-bird-species/train',target_size=(224, 224),batch_size=32)
valid_ds = generator.flow_from_directory('../input/100-bird-species/valid',target_size=(224, 224),batch_size=32)
model.compile(optimizer=keras.optimizers.Adam(1e-4),loss='categorical_crossentropy',metrics=['accuracy'])
h1=model.fit(train_ds,epochs= 20,validation_data=valid_ds,workers=10,use_multiprocessing=True)

In [2]:
test_ds = generator.flow_from_directory('../input/100-bird-species/test',target_size=(224, 224),batch_size=32)
test_loss, test_acc = model.evaluate(test_ds)
print('test acc: ', round(test_acc*100, 2),  '%')

In [12]:
import matplotlib.pyplot as plt
def plot_losses(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
#     train_losses = [x['train_loss'] for x in history.history]
#     val_losses = [x['val_loss'] for x in history.history]
    plt.plot(epochs,acc, '-bx')
    plt.plot(epochs,val_acc, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

plot_losses(h1)

In [13]:
model.save('my_model.h5',overwrite=True)

In [14]:
model.compile(optimizer=keras.optimizers.Adam(1e-5),loss='categorical_crossentropy',metrics=['accuracy'])
h1=model.fit(train_ds,epochs= 10,validation_data=valid_ds,workers=10,use_multiprocessing=True)

In [17]:
model.compile(optimizer=keras.optimizers.Adam(2e-6),loss='categorical_crossentropy',metrics=['accuracy'])
h1=model.fit(train_ds,epochs= 5,validation_data=valid_ds,workers=10,use_multiprocessing=True)

In [18]:
model.save('my_model1.h5',overwrite=True)

In [19]:
test_ds = generator.flow_from_directory('../input/100-bird-species/test',target_size=(224, 224),batch_size=32)
test_loss, test_acc = model.evaluate(test_ds)
print('test acc: ', round(test_acc*100, 2),  '%')

In [ ]:
<a href="./my_model1.h5"> Download File </a>

<a href="./my_model1.h5"> Download File </a>